# 1.Generative Models for Text
Ruotian Jiang 8389636738

# a & b & c (i)

In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
data3 = open('3.txt', 'r').read()
data4 = open('4.txt', 'r').read()
data1 = open('1.txt', 'r').read()
data2 = open('2.txt', 'r').read()
data = data1 + data2 + data3 + data4
data = data.lower()
# here to disregard lowercase and uppercase letters.

n_chars = len(data)
n_vocab = len(set(data))
print "Total Characters: ", n_chars
print "Total Vocab: ", n_vocab

/usr/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Total Characters:  1624629
Total Vocab:  100


# c (ii)

In [2]:
data_ascii = [ord(i) for i in data]
min_v = min(data_ascii)
max_v = max(data_ascii)

# c(iii) & c(iv)

In [3]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = data_ascii[i:i + seq_length]
    seq_out = data_ascii[i + seq_length]
    dataX.append([j for j in seq_in])
    dataY.append(seq_out)
n_patterns = len(dataX)
print "Total Patterns: ", n_patterns

Total Patterns:  1624529


# c(v)

In [4]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# rescaled to [1:0]
X = (X - min_v)/float(max_v-min_v)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
print y

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# c(vi) & c(vii) & c(viii) & c(ix) & c(x) 
Because of the slowness and because of our optimization requirements, we will use model checkpointing to record all of the network weights to file each time an improvement in loss is observed at the end of the epoch. We will use the best set of weights (lowest loss) to instantiate our generative model in the next section.
Because 30 for epochs is time cosuming, about 40 hours, so I choose 20 here.

In [6]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
1624529/1624529 [==============================] - 4701s 3ms/step - loss: 2.7764

Epoch 00001: loss improved from inf to 2.77635, saving model to weights-improvement-01-2.7764.hdf5
Epoch 2/20
1624529/1624529 [==============================] - 4707s 3ms/step - loss: 2.5714

Epoch 00002: loss improved from 2.77635 to 2.57143, saving model to weights-improvement-02-2.5714.hdf5
Epoch 3/20
1624529/1624529 [==============================] - 4719s 3ms/step - loss: 2.4649

Epoch 00003: loss improved from 2.57143 to 2.46494, saving model to weights-improvement-03-2.4649.hdf5
Epoch 4/20
1624529/1624529 [==============================] - 4731s 3ms/step - loss: 2.3721

Epoch 00004: loss improved from 2.46494 to 2.37211, saving model to weights-improvement-04-2.3721.hdf5
Epoch 5/20
1624529/1624529 [==============================] - 4738s 3ms/step - loss: 2.2939

Epoch 00005: loss improved from 2.37211 to 2.29392, saving model to weights-improvement-05-2.2939.hdf5
Epoch 6/20
1624529/16245

In [19]:
# Load LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
# load ascii text and covert to lowercase
text = 'There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object.'
text = text.lower()
# create mapping of unique chars to integers, and a reverse mapping

# summarize the loaded data
n_chars = len(text)
n_vocab = len(set(text))
print "Total Characters: ", n_chars
print "Total Vocab: ", n_vocab

text_ascii = [ord(i) for i in text]
min_t = min(text_ascii)
max_t = max(text_ascii)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = text_ascii[i:i + seq_length]
    seq_out = text_ascii[i + seq_length]
    dataX.append([j for j in seq_in])
    dataY.append(seq_out)
n_patterns = len(dataX)
print "Total Patterns: ", n_patterns
# load the network weights
filename = "weights-improvement-19-1.9348.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print "original:"
print "\"", ''.join([chr(value) for value in pattern]), "\""
# generate characters
signal = 0 
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = (x - min_t)/float(max_t-min_t)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = chr(index)
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print "\nDone."

Total Characters:  153
Total Vocab:  27
Total Patterns:  53
original:
" ively, just as they would physical phenomena. this school of psychologists tends not to emphasize th "
uh 



















































































?

ihu thuh 

ehs 

phu 

thm 

ph thmhi,h��e��,e��e��e,i��h��e: 



























































































?
i��h,ah thmhu,t��e��enipeni 

ehs 






















































































?
ah th ih th 

a��h��en 





























































































a��hle 










































































?
ais hhuhs�ivin,�� thuhs,hiu� thuhu��,
th 






















































































ae sh
ih hh th 















































































ae sh hh th 

eh th 













In [14]:
chr(32)

' '